In [ ]:
import pandas as pd
import re

# Load the scraped job data
df = pd.read_csv("data/indeed_scraped_job_details.csv")



,job_id,url,scraped_content
0,a1944017870e5468,https://www.indeed.com/viewjob?jk=a1944017870e...,[Indeed Home](https://www.indeed.com/?from=gna...
1,05bb5030f12e78bf,https://www.indeed.com/viewjob?jk=05bb5030f12e...,Skip to main content [Indeed Home](https://www...
2,7eebbb74fa3b5d15,https://www.indeed.com/viewjob?jk=7eebbb74fa3b...,Skip to main content [Indeed Home](https://www...
3,67870a316a3a0356,https://www.indeed.com/viewjob?jk=67870a316a3a...,[Indeed Home](https://www.indeed.com/?from=gna...
4,c92dd0299d407173,https://www.indeed.com/viewjob?jk=c92dd0299d40...,Skip to main content [Indeed Home](https://www...


In [ ]:
# Helper function to extract fields
def extract_fields(text):
    # Job Title
    title_match = re.search(r"#\s*(.*?)\n", text)
    job_title = title_match.group(1).strip() if title_match else None

    # Company Name
    company_match = re.search(r"\[(.*?)\]\(https://www\.indeed\.com/cmp/", text)
    company = company_match.group(1).strip() if company_match else None

    # Location
    location_match = re.search(r"\d+\s+.*?,\s+[A-Z]{2}\s+\d{5}", text)
    location = location_match.group(0).strip() if location_match else None

    # Salary
    salary_match = re.search(r"\$[\d,]+ - \$[\d,]+", text)
    salary = salary_match.group(0).strip() if salary_match else None

    return pd.Series([job_title, company, location, salary])

# Apply to dataframe
df[['job_title', 'company', 'location', 'salary']] = df['scraped_content'].apply(extract_fields)

# Preview the result
df[['job_title', 'company', 'location', 'salary']].head()


,job_title,company,location,salary
0,"Senior Data Scientist, 5+ Years of Experience",Snapchat,"63 Market Street, Los Angeles, CA 90291","$162,000 - $270,000"
1,"Machine Learning Engineer, Level 4",Snapchat,None,"$147,000 - $259,000"
2,"# Machine Learning Engineer, Level 5",Snapchat,None,"$178,000 - $313,000"
3,"Machine Learning Engineer, Generative AI",Snapchat,"63 Market Street, Los Angeles, CA 90291","$100,000 - $176,000"
4,"Staff Software Engineer, ML Infrastructure, AI...",Snapchat,"63 Market Street, Los Angeles, CA 90291","$195,000 - $343,000"


In [ ]:
from sqlalchemy import create_engine
import os

# Replace with your actual credentials or use environment variables
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")


# Create engine
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")


In [ ]:
df[['job_id', 'url', 'job_title', 'company', 'location', 'salary']].to_sql(
    "fact_indeed_jobs",
    con=engine,
    if_exists="replace",
    index=False
)


10